In [44]:
%load_ext autoreload
%autoreload 2
import numpy as np
import os
import pickle
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score, roc_auc_score
import sys
sys.path.append(os.path.abspath(".."))
import torch

from datasets import PressingSequenceDataset, SoccerMapInputDataset

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# Normalization
num_features = train_dataset[0]['features'].shape[2]

# 각 피처 차원별 최솟값과 최댓값을 저장할 텐서를 초기화합니다.
# shape: [NumFeatures]
current_min_vals = torch.full((num_features,), float('inf'), dtype=torch.float32)
current_max_vals = torch.full((num_features,), float('-inf'), dtype=torch.float32)

for idx in range(len(train_dataset)):
    features_tensor = train_dataset[0]['features']
    min_in_tensor = torch.min(torch.min(features_tensor, dim=0).values, dim=0).values
    max_in_tensor = torch.max(torch.max(features_tensor, dim=0).values, dim=0).values

    current_min_vals = torch.minimum(current_min_vals, min_in_tensor)
    current_max_vals = torch.maximum(current_max_vals, max_in_tensor)

self.feature_min_vals = current_min_vals
self.feature_max_vals = current_max_vals

# __getitem__에서 브로드캐스팅을 위해 shape 변경: [1, 1, NumFeatures]
self.min_vals_bcast = self.feature_min_vals.reshape(1, 1, -1)
self.max_vals_bcast = self.feature_max_vals.reshape(1, 1, -1)

self.feature_ranges = self.max_vals_bcast - self.min_vals_bcast
# 범위가 0인 경우 (피처 값이 상수인 경우) 0으로 나누는 것을 방지합니다.
# 이 경우 정규화된 값은 0이 됩니다. ( (X - min) / 1 = 0 )
self.feature_ranges[self.feature_ranges == 0] = 1.0 

In [45]:
data_path = "/data/MHL/pressing-intensity"

with open(f"{data_path}/train_dataset.pkl", "rb") as f:
    train_dataset = pickle.load(f)

with open(f"{data_path}/test_dataset.pkl", "rb") as f:
    test_dataset = pickle.load(f)

len(train_dataset), len(test_dataset)

(14241, 1721)

In [3]:
sample = train_dataset[0]
sample.keys()

dict_keys(['features', 'pressing_intensity', 'label', 'presser_id', 'agent_order'])

In [4]:
print(f"Features : {sample['features'].shape}")
print(f"Pressing Intensity : {sample['pressing_intensity'].shape}")
print(f"Labels : {sample['label']}")
print(f"Presser ID : {sample['presser_id']}")
print(f"Players Order : {sample['agent_order']}")

Features : torch.Size([150, 23, 8])
Pressing Intensity : torch.Size([150, 11, 11])
Labels : 0
Presser ID : 77414
Players Order : ['188178', '250079', '250101', '250102', '500133', '500140', '500141', '500142', '62365', '62386', '77414', '187259', '343587', '408792', '500113', '500115', '500116', '500117', '500118', '500121', '500502', '83615', 'ball']


In [5]:
x_tensor = sample['features']
press_intensity = sample['pressing_intensity']
y_tensor = sample['label']
x_tensor.shape, press_intensity.shape, y_tensor

(torch.Size([150, 23, 8]), torch.Size([150, 11, 11]), tensor(0))

In [6]:
x_tensor_lst = [sample['features'] for sample in train_dataset]
x_tensor_lst = torch.cat(x_tensor_lst)
feature_cols = ['x', 'y', 'vx', 'vy', 'v', 'ax', 'ay', 'a']

for i in range(8):
    print(f"{feature_cols[i]} : {x_tensor_lst[..., i].min()} ~ {x_tensor_lst[..., i].max()}")

x : -58.351173400878906 ~ 57.75
y : -38.8927001953125 ~ 38.8927001953125
vx : -342.4437561035156 ~ 301.3289489746094
vy : -200.180419921875 ~ 280.8352966308594
v : 0.0 ~ 28.0
ax : -11283.9609375 ~ 9192.126953125
ay : -5758.1103515625 ~ 8190.6416015625
a : 0.0 ~ 13.5


# 1. XGBoost

In [10]:
num_seq = 150
num_agents = 11
use_pressing_intensity = False
selected_features_idx = [1, 2, 4, 7]

train_features = []
train_labels = []
for i in range(len(train_dataset)):
    sample = train_dataset[i]
    x_tensor = sample['features'][..., selected_features_idx][-90:]
    press_intensity = sample['pressing_intensity'][-90:]
    y_tensor = sample['label']
    
    # Flatten the sequence data: shape (sequence_length, num_features) -> (sequence_length*num_features,)
    feature_vector = x_tensor.flatten().numpy()
    if use_pressing_intensity:
        if press_intensity.shape[1] != num_agents:
            pad_tensor = torch.zeros(press_intensity.shape[0], num_agents-press_intensity.shape[1], press_intensity.shape[2])
            press_intensity = torch.cat([press_intensity, pad_tensor], dim=1)
            
        if press_intensity.shape[2] != num_agents:
            pad_tensor = torch.zeros(press_intensity.shape[0], press_intensity.shape[1], num_agents-press_intensity.shape[2])
            press_intensity = torch.cat([press_intensity, pad_tensor], dim=2)
        
        press_vector = press_intensity.flatten().numpy()
        # Concatenate the flattened sequence data with the pressing intensity
        feature_vector = np.concatenate((feature_vector, press_vector))
    train_features.append(feature_vector)
    train_labels.append(y_tensor.item())


test_features = []
test_labels = []
for i in range(len(test_dataset)):
    sample = test_dataset[i]
    x_tensor = sample['features'][..., selected_features_idx][-90:]
    press_intensity = sample['pressing_intensity'][-90:]
    y_tensor = sample['label']
    
    # Flatten the sequence data: shape (sequence_length, num_features) -> (sequence_length*num_features,)
    feature_vector = x_tensor.flatten().numpy()
    
    if use_pressing_intensity:
        if press_intensity.shape[1] != num_agents:
            pad_tensor = torch.zeros(press_intensity.shape[0], num_agents-press_intensity.shape[1], press_intensity.shape[2])
            press_intensity = torch.cat([press_intensity, pad_tensor], dim=1)
        if press_intensity.shape[2] != num_agents:
            pad_tensor = torch.zeros(press_intensity.shape[0], press_intensity.shape[1], num_agents-press_intensity.shape[2])
            press_intensity = torch.cat([press_intensity, pad_tensor], dim=2)
        
        press_vector = press_intensity.flatten().numpy()
        # Concatenate the flattened sequence data with the pressing intensity
        feature_vector = np.concatenate((feature_vector, press_vector))
    test_features.append(feature_vector)
    test_labels.append(y_tensor.item())

X_train = np.array(train_features)
y_train = np.array(train_labels)

# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

X_test = np.array(test_features)
y_test = np.array(test_labels)

In [12]:
def print_dataset_distribution(y_train, y_val, y_test):
    def _print_split(name, labels):
        total = len(labels)
        unique, counts = np.unique(labels, return_counts=True)
        print(f"{name} Set:")
        print(f"  Total samples: {total}")
        for label, count in zip(unique, counts):
            percent = (count / total) * 100
            print(f"    Label {label}: {count:>5} samples ({percent:5.2f}%)")
        print("-" * 40)

    print("\n📊 Dataset Distribution Summary")
    print("=" * 40)
    _print_split("Train", y_train)
    _print_split("Validation", y_val)
    _print_split("Test", y_test)

print_dataset_distribution(y_train, y_val, y_test)


📊 Dataset Distribution Summary
Train Set:
  Total samples: 11392
    Label 0:  9138 samples (80.21%)
    Label 1:  2254 samples (19.79%)
----------------------------------------
Validation Set:
  Total samples: 2849
    Label 0:  2281 samples (80.06%)
    Label 1:   568 samples (19.94%)
----------------------------------------
Test Set:
  Total samples: 1721
    Label 0:  1358 samples (78.91%)
    Label 1:   363 samples (21.09%)
----------------------------------------


In [9]:
# Create XGBoost DMatrix objects for train and test sets
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_val, label=y_val)

# Set XGBoost training parameters
params = {
    'objective': 'binary:logistic',  # binary classification
    'eval_metric': 'auc',            # evaluation metric: AUC
    'max_depth': 6,                  # maximum depth of trees
    'eta': 0.1,                      # learning rate
    'seed': 42
}

# Specify the watchlist to evaluate performance on training and test sets during training
watchlist = [(dtrain, 'train'), (dtest, 'eval')]
num_rounds = 100

In [10]:
# Train XGBoost model with early stopping on the evaluation set
bst = xgb.train(params, dtrain, num_rounds, watchlist, early_stopping_rounds=10)

/home/work/miniconda/envs/mhl_py311/lib/python3.11/site-packages/xgboost/core.py:726: FutureWarning: Pass `evals` as keyword args.
  warnings.warn(msg, FutureWarning)


[0]	train-auc:0.72404	eval-auc:0.59755
[1]	train-auc:0.77449	eval-auc:0.61684
[2]	train-auc:0.80143	eval-auc:0.61842
[3]	train-auc:0.82563	eval-auc:0.62081
[4]	train-auc:0.83993	eval-auc:0.62739
[5]	train-auc:0.85611	eval-auc:0.63009
[6]	train-auc:0.86313	eval-auc:0.63397
[7]	train-auc:0.87409	eval-auc:0.63416
[8]	train-auc:0.88935	eval-auc:0.63792
[9]	train-auc:0.89944	eval-auc:0.63648
[10]	train-auc:0.90767	eval-auc:0.63826
[11]	train-auc:0.91626	eval-auc:0.63678
[12]	train-auc:0.92196	eval-auc:0.63293
[13]	train-auc:0.92891	eval-auc:0.63344
[14]	train-auc:0.93481	eval-auc:0.63360
[15]	train-auc:0.93753	eval-auc:0.63349
[16]	train-auc:0.94254	eval-auc:0.63322
[17]	train-auc:0.94739	eval-auc:0.63347
[18]	train-auc:0.94938	eval-auc:0.63573
[19]	train-auc:0.95371	eval-auc:0.63360
[20]	train-auc:0.95596	eval-auc:0.63550


In [ ]:
# W/O Pressing Intensity
# Get predictions on the test set
y_pred = bst.predict(dtest)
y_pred_label = (y_pred > 0.5).astype(int)

# Calculate and print evaluation metrics
accuracy = accuracy_score(y_val, y_pred_label)
auc = roc_auc_score(y_val, y_pred)

print("Test Accuracy: {:.4f}".format(accuracy))
print("Test AUC: {:.4f}".format(auc))

Test Accuracy: 0.8027
Test AUC: 0.6491


: 

In [11]:
# W/O Pressing Intensity
# Get predictions on the test set
y_pred = bst.predict(dtest)
y_pred_label = (y_pred > 0.5).astype(int)

# Calculate and print evaluation metrics
accuracy = accuracy_score(y_val, y_pred_label)
auc = roc_auc_score(y_val, y_pred)

print("Test Accuracy: {:.4f}".format(accuracy))
print("Test AUC: {:.4f}".format(auc))

Test Accuracy: 0.7989
Test AUC: 0.6465


# 2. SoccerMap / exPress Evaluation

In [39]:
import torch
import pytorch_lightning as pl
from torch.utils.data import DataLoader
import json
import os
os.chdir('/home/work/MHL/express-v2')
import argparse # To accept checkpoint path as argument

# Import project modules
# import config  # Import static configurations
from model import PytorchSoccerMapModel # Import Lightning model
from datasets import PressingSequenceDataset, SoccerMapInputDataset 


In [40]:
pl.seed_everything(42, workers=True) # Ensure reproducibility

DATA_PATH = "/data/MHL/pressing-intensity" # Path where pickled datasets are saved
test_dataset = SoccerMapInputDataset(os.path.join(DATA_PATH, "test_dataset.pkl"))

if len(test_dataset) == 0:
    print("Loaded test dataset is empty. Exiting.")

# Custom collate function to handle potential None values from dataset errors
def collate_fn_skip_none(batch):
    batch = list(filter(lambda x: x is not None, batch))
    if not batch: return None
    try: return torch.utils.data.dataloader.default_collate(batch)
    except RuntimeError: return None # Skip batch if collation error

test_loader = DataLoader(
    test_dataset,
    batch_size=1,
    shuffle=False,
    num_workers=4,
    pin_memory=True,
    persistent_workers=True,
    # collate_fn=collate_fn_skip_none
)

Seed set to 42


Loading dataset from /data/MHL/pressing-intensity/test_dataset.pkl...


In [41]:
import argparse

parser = argparse.ArgumentParser(description="Train a pressing evaluation model.")
# parser.add_argument("--model_type", type=str, default="soccermap", choices=['soccermap', 'xgboost', 'exPress'], help="Path to the model checkpoint (.ckpt) file saved during training.")
# parser.add_argument("--root_path", type=str, default="/data/MHL/pressing-intensity", help="Path to the data file.")
parser.add_argument("--model_type", type=str, default="soccermap", choices=['soccermap', 'xgboost', 'exPress'], help="Path to the model checkpoint (.ckpt) file saved during training.")
parser.add_argument("--root_path", type=str, default="/data/MHL/pressing-intensity", help="Path to the data file.")
parser.add_argument("--mode", type=str, default="train", choices=['train', 'test'], help="Mode: 'train' or 'test'.")
parser.add_argument("--ckpt_path", type=str, default=None, help="Path to checkpoint file (Required for 'test' mode).")
parser.add_argument("--params_path", type=str, default="params.json", help="Path to the JSON containing configurations.")
parser.add_argument("--seed", type=int, default=42, help="Seed number.")

args = parser.parse_args([])

args.mode = 'test'
args.model_type = "exPress"
args.ckpt_path = "/data/MHL/pressing-intensity/checkpoints/exPress-epoch=28-val_loss=0.49.ckpt"

In [42]:
from components import press


component_dict = {
                    "soccermap": press.SoccerMapComponent,
                    "exPress": press.exPressComponent,
                }

exp = component_dict[args.model_type](args)

Seed set to 42


Configurations loaded from params.json.
